In [1]:
# Import des modules généraux
import pandas as pd
import re
from collections import Counter
from dotenv import load_dotenv
import os

# Import des modules de NLP
from bertopic import BERTopic
from bertopic.representation import KeyBERTInspired, MaximalMarginalRelevance, OpenAI, PartOfSpeech
from bertopic.vectorizers import ClassTfidfTransformer

import spacy
from spacy.language import Language
from spacy_language_detection import LanguageDetector

from nltk.tokenize import sent_tokenize, word_tokenize
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import CountVectorizer

# Import des modules de clustering
from hdbscan import HDBSCAN
from umap import UMAP

# Import des modules de visualisation
import matplotlib.pyplot as plt
import seaborn as sns

# Import de openai pour la représentation des topics améliorée
import openai

# Chargement des variables d'environnement et initialisations
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")
nlp = spacy.load("fr_core_news_md")

def get_lang_detector(nlp, name):
    return LanguageDetector(seed=42)  # We use the seed 42

Language.factory("language_detector", func=get_lang_detector)
nlp.add_pipe('language_detector', last=True)

# Texte complet contenant toutes les lettres (dans data/lettres.txt)
text  = open("data/lettres.txt", "r", encoding="utf-8").read()

# Expression régulière pour trouver les lettres
pattern = r"Lettre \d+ : \d{2}\.\d{2}\.\d{4}.*?(?=Lettre \d+ : \d{2}\.\d{2}\.\d{4}|$)"

# Trouver toutes les occurrences des lettres
letters = re.findall(pattern, text, re.DOTALL)

# Créer un DataFrame avec une ligne par lettre
df = pd.DataFrame(letters, columns=["letter"])

# Expression régulière pour trouver les dates
date_pattern = r'(\d{2}\.\d{2}\.\d{4})'

# Extraire les dates de la colonne 'letter'
df['date'] = df['letter'].str.extract(date_pattern, expand=False)

# créer une colonne year
df['year'] = df['date'].str.extract(r'(\d{4})', expand=False)

# trier par année
df = df.sort_values('year')
df

,letter,date,year
70,Lettre 1 : 31.03.2004\nLettre 1\n31.03.2004\n\...,31.03.2004,2004
67,"Lettre 8 : 08.11.2004\nChers collègues, chers ...",08.11.2004,2004
69,Lettre 2 : 05.04.2004\nLettre 2\n05.04.2004\n\...,05.04.2004,2004
68,"Lettre 7 : 14.09.2004\nChers collègues, chers ...",14.09.2004,2004
66,"Lettre 9 : 24.01.2005\nChers collègues, chers ...",24.01.2005,2005
...,...,...,...
4,Lettre 75 : 11.12.2018\n\nIntégrité académique...,11.12.2018,2018
3,Lettre 76 : 21.03.2019\n\nIRAFPA\tInstitut de ...,21.03.2019,2019
2,Lettre 77 : 27.05.2019\n\nIRAFPA\tInstitute fo...,27.05.2019,2019
1,Lettre 78 : 24.10.2019\n\n\n\nIRAFPA\t Institu...,24.10.2019,2019


In [2]:
# Initialize an empty list to store the rows
data = []

for index, row in df.iterrows():
    date = row["date"]
    content = row["letter"]
    
    # Tokenize the content into sentences
    sentences = sent_tokenize(content)

    # Enlever les caractères spéciaux, les espaces en trop et mettre en minuscule
    #sentences = [re.sub(r"[^a-zA-Z0-9\s]", "", sentence) for sentence in sentences]
    sentences = [" ".join(sentence.split()) for sentence in sentences]
    sentences = [sentence.lower() for sentence in sentences]
    
    # Enlever les phrases qui ont moins de 16 mots
    sentences = [sentence for sentence in sentences if len(word_tokenize(sentence)) > 16]

    # Enlever les phrases en langue étrangère
    sentences = [sentence for sentence in sentences if nlp(sentence)._.language["language"] == "fr"]

    # Enlever les phrases qui ont plus de 10 caractères spéciaux
    #sentences = [sentence for sentence in sentences if len(re.findall(r"[^a-zA-Z0-9\s]", sentence)) < 10]

    # Enlever les phrases contenant "operator"
    #sentences = [sentence for sentence in sentences if "operator" not in sentence.lower()]

    # Enlever les " --" à l'intérieur des phrases
    #sentences = [sentence.replace(" --", "") for sentence in sentences]

    # Enlever les phrases en double
    sentences = list(set(sentences))
    
    # Append each sentence with its date to the list
    for sentence in sentences:
        data.append([date, sentence])

# Create a new DataFrame from the accumulated data
df_dates_sentences = pd.DataFrame(data, columns=["Date", "Sentence"])

# Supprimer les phrases en double
df_dates_sentences = df_dates_sentences.drop_duplicates(subset=["Sentence"])

df_dates_sentences

,Date,Sentence
0,31.03.2004,les cinq avaient emprunté plus de 95% de leur ...
1,31.03.2004,- deux de nos assistants doctorants ont été tr...
2,31.03.2004,"origine grecque: deon et logos, soit le devoir..."
3,31.03.2004,si vous connaissez des collègues qui travaille...
4,31.03.2004,je rêve qu'ensemble nous développions et appli...
...,...,...
1292,14.01.2020,nous avons besoin de votre contribution pour l...
1293,14.01.2020,les résultats de ces deux recherches seront pr...
1294,14.01.2020,ces recherches permettront aux dirigeants d’éc...
1295,14.01.2020,merci de me contacter directement par courriel...


In [3]:
# Print le nombre de mots par phrase en moyenne avec l'écart-type
print(f"Moyenne: {df_dates_sentences['Sentence'].str.split().str.len().mean()}")
print(f"Ecart-type: {df_dates_sentences['Sentence'].str.split().str.len().std()}")

dates = df_dates_sentences["Date"].tolist()
sentences = df_dates_sentences["Sentence"].tolist()

# Print the length of the dates and sentences
print(f"\nNumber of dates: {len(dates)}")
print(f"Number of sentences: {len(sentences)}")

Moyenne: 27.369620253164555
Ecart-type: 11.442205918942419

Number of dates: 1185
Number of sentences: 1185


In [4]:
# Pre-calculate embeddings
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = embedding_model.encode(sentences, show_progress_bar=True)

Batches:   0%|          | 0/38 [00:00<?, ?it/s]

In [5]:
umap_model = UMAP(n_neighbors=15, n_components=5, min_dist=0.0, metric='cosine', random_state=42)
"""
hdbscan_model = HDBSCAN(min_cluster_size=40, metric='euclidean', cluster_selection_method='eom', prediction_data=True)
vectorizer_model = CountVectorizer(stop_words="french", min_df=2, ngram_range=(1, 2))
ctfidf_model = ClassTfidfTransformer(reduce_frequent_words=False, bm25_weighting=False)
"""

'\nhdbscan_model = HDBSCAN(min_cluster_size=40, metric=\'euclidean\', cluster_selection_method=\'eom\', prediction_data=True)\nvectorizer_model = CountVectorizer(stop_words="french", min_df=2, ngram_range=(1, 2))\nctfidf_model = ClassTfidfTransformer(reduce_frequent_words=False, bm25_weighting=False)\n'

In [6]:
# KeyBERT
keybert_model = KeyBERTInspired()

# Part-of-Speech
pos_model = PartOfSpeech("en_core_web_sm")

# MMR
mmr_model = MaximalMarginalRelevance(diversity=0.3)

# GPT-3.5 Turbo
prompt = """
I have a topic that contains the following documents:
[DOCUMENTS]
The topic is described by the following keywords: [KEYWORDS]

En se basant sur les information ci-dessus, extrais un titre très descriptif du topic en français de maximum 7 mots. Make sure it is in the following format:
topic: <topic label>
"""

client = openai.OpenAI(api_key=openai_api_key)
openai_model = OpenAI(client,
                       model="gpt-3.5-turbo",
                       exponential_backoff=True, 
                       chat=True, 
                       nr_docs=5,
                       prompt=prompt)

# Création du modèle de topic modeling
representation_model = {
    "KeyBERT": keybert_model,
    "OpenAI": openai_model,
    "MMR": mmr_model,
    "POS": pos_model
}

In [7]:
topic_model = BERTopic(

  # Pipeline models
  embedding_model=embedding_model,
  umap_model=umap_model,
  #hdbscan_model=hdbscan_model,
  #vectorizer_model=vectorizer_model,
  representation_model=representation_model,
  #ctfidf_model=ctfidf_model,

  # Hyperparameters
  #language="french",
  #top_n_words=10,
  #min_topic_size=70,
  verbose=True
)

topics, probs = topic_model.fit_transform(sentences, embeddings)

2024-03-11 16:45:38,403 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-03-11 16:45:45,575 - BERTopic - Dimensionality - Completed ✓
2024-03-11 16:45:45,576 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-03-11 16:45:45,610 - BERTopic - Cluster - Completed ✓
2024-03-11 16:45:45,615 - BERTopic - Representation - Extracting topics from clusters using representation models.
100%|██████████| 7/7 [00:05<00:00,  1.28it/s]
2024-03-11 16:45:53,587 - BERTopic - Representation - Completed ✓


In [8]:
gpt_topic_labels = {topic: " | ".join(list(zip(*values))[0]) for topic, values in topic_model.topic_aspects_["OpenAI"].items()}
gpt_topic_labels[-1] = "Outlier Topic"
topic_model.set_topic_labels(gpt_topic_labels)

In [9]:
# Reduce dimensionality of embeddings, this step is optional but much faster to perform iteratively:
reduced_embeddings = UMAP(metric='cosine', random_state=42).fit_transform(embeddings)

In [10]:
# Visualize the documents in 2-dimensional space and show the titles on hover instead of the abstracts
fig = topic_model.visualize_documents(sentences,
                                reduced_embeddings=reduced_embeddings,
                                custom_labels=True)
fig

In [11]:
list_of_topics = topic_model.get_topic_info()
list_of_topics

,Topic,Count,Name,CustomName,Representation,KeyBERT,OpenAI,MMR,POS,Representative_Docs
0,-1,20,-1_de_médiation_et_des,Outlier Topic,"[de, médiation, et, des, notre, une, ne, adhés...","[médiation, une, entre, scientifique, établiss...",[Médiation et adhésions à une association],"[de, médiation, et, des, notre, une, ne, adhés...","[médiation, et, des, adhésions, responsabilité...",[pour ceux qui ne l’ont pas encore fait : les ...
1,0,966,0_de_et_la_des,Plagiats et fraudes universitaires graves,"[de, et, la, des, le, les, en, un, nous, que]","[plagieur, votre, scientifique, une, plagié, ê...",[Plagiats et fraudes universitaires graves],"[de, et, la, des, le, les, en, un, nous, que]","[et, nous, que, vous, tous, si, ceux, avons, c...",[• que la lecture de ce cas soit recommandée à...
2,1,71,1_chers_lettre_collègues_étudiants,Correspondance sur l'éducation universitaire,"[chers, lettre, collègues, étudiants, lecteurs...","[une, lecteurs, chers, pas, ceux, lettre, le, ...",[Correspondance sur l'éducation universitaire],"[chers, lettre, collègues, étudiants, lecteurs...","[chers, lettre, collègues, étudiants, lecteurs...","[2011 chers collègues, chers étudiants, chers ..."
3,2,64,2_fraude_de_et_la,Fraude et plagiat académiques en action,"[fraude, de, et, la, plagiat, le, sur, action,...","[fraude, fraudes, fraudeurs, plagiat, scientif...",[Fraude et plagiat académiques en action],"[fraude, de, et, la, plagiat, le, sur, action,...","[fraude, et, plagiat, action, académiques, che...","[michelle bergadaà, mba, ph.d. présidente de l..."
4,3,29,3_responsable_site_collaboratif_contributions,Contributions régulières via site responsable ...,"[responsable, site, collaboratif, contribution...","[responsable, responsabilité, merci, une, coll...",[Contributions régulières via site responsable...,"[responsable, site, collaboratif, contribution...","[responsable, site, contributions, vos, très, ...",[un tout grand merci pour vos contributions ré...
5,4,24,4_michelle_bergadaà_genève_professeur,"Michelle Bergadaà, professeur marketing Genève","[michelle, bergadaà, genève, professeur, de, m...","[marketing, communication, commun, stratégies,...","[Michelle Bergadaà, professeur marketing Genève]","[michelle, bergadaà, genève, professeur, de, m...","[marketing, université, ovsm, et, communicatio...",[michelle bergadaà professeur de communication...
6,5,11,5_chef_rédacteur_revue_ce,Problème d'éthique dans revue scientifique,"[chef, rédacteur, revue, ce, en, de, rédacteur...","[rédacteur, rédacteurs, réfléchissent, être, p...",[Problème d'éthique dans revue scientifique],"[chef, rédacteur, revue, ce, en, de, rédacteur...","[chef, excuses, dans, revues, lettre, siennes,...",[l’auteur floué va demander formellement au ré...


# Topic modeling 2

In [49]:
# Ne garder que les topics -1, 0, 2, 3, 5 qui sont les plus intéressants
df_dates_sentences_2 = df_dates_sentences.copy()
df_dates_sentences_2["Topic"] = topics
df_dates_sentences_2 = df_dates_sentences_2[df_dates_sentences_2["Topic"].isin([-1, 0, 2, 3, 5])]
df_dates_sentences_2["Year"] = df_dates_sentences_2["Date"].str.extract(r'(\d{4})', expand=False)
df_dates_sentences_2

,Date,Sentence,Topic,Year
0,31.03.2004,les cinq avaient emprunté plus de 95% de leur ...,0,2004
1,31.03.2004,- deux de nos assistants doctorants ont été tr...,2,2004
2,31.03.2004,"origine grecque: deon et logos, soit le devoir...",0,2004
3,31.03.2004,si vous connaissez des collègues qui travaille...,0,2004
4,31.03.2004,je rêve qu'ensemble nous développions et appli...,0,2004
...,...,...,...,...
1292,14.01.2020,nous avons besoin de votre contribution pour l...,0,2020
1293,14.01.2020,les résultats de ces deux recherches seront pr...,0,2020
1294,14.01.2020,ces recherches permettront aux dirigeants d’éc...,0,2020
1295,14.01.2020,merci de me contacter directement par courriel...,0,2020


In [50]:

# Print la longueur des dates et des phrases
print(f"\nNumber of dates: {len(df_dates_sentences_2['Date'].tolist())}")
print(f"Number of sentences: {len(df_dates_sentences_2['Sentence'].tolist())}")

# Pré-calculer les embeddings
sentences_2 = df_dates_sentences_2["Sentence"].tolist()
dates_2 = df_dates_sentences_2["Year"].tolist()
embeddings_2 = embedding_model.encode(sentences_2, show_progress_bar=True)

topic_model_2 = BERTopic(
    
      # Pipeline models
      embedding_model=embedding_model,
      #umap_model=umap_model,
      #hdbscan_model=hdbscan_model,
      #vectorizer_model=vectorizer_model,
      representation_model=representation_model,
      #ctfidf_model=ctfidf_model,
    
      # Hyperparameters
      #language="french",
      #top_n_words=10,
      min_topic_size=5,
      verbose=True
    )

topics_2, probs_2 = topic_model_2.fit_transform(sentences_2, embeddings_2)

gpt_topic_labels_2 = {topic: " | ".join(list(zip(*values))[0]) for topic, values in topic_model_2.topic_aspects_["OpenAI"].items()}
gpt_topic_labels_2[-1] = "Outlier Topic"
topic_model_2.set_topic_labels(gpt_topic_labels_2)

# Reduce dimensionality of embeddings, this step is optional but much faster to perform iteratively:
reduced_embeddings_2 = UMAP(metric='cosine', random_state=42).fit_transform(embeddings_2)

# Visualize the documents in 2-dimensional space and show the titles on hover instead of the abstracts
fig = topic_model_2.visualize_documents(sentences_2,
                                reduced_embeddings=reduced_embeddings_2,
                                custom_labels=True)
fig


Number of dates: 1090
Number of sentences: 1090


Batches:   0%|          | 0/35 [00:00<?, ?it/s]

2024-03-11 17:04:13,348 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-03-11 17:04:16,065 - BERTopic - Dimensionality - Completed ✓
2024-03-11 17:04:16,066 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-03-11 17:04:16,102 - BERTopic - Cluster - Completed ✓
2024-03-11 17:04:16,104 - BERTopic - Representation - Extracting topics from clusters using representation models.
100%|██████████| 37/37 [00:27<00:00,  1.35it/s]
2024-03-11 17:04:51,849 - BERTopic - Representation - Completed ✓


In [51]:
topic_model_2.get_topic_info()

,Topic,Count,Name,CustomName,Representation,KeyBERT,OpenAI,MMR,POS,Representative_Docs
0,-1,315,-1_de_et_des_la,Outlier Topic,"[de, et, des, la, un, en, le, vous, les, que]","[une, votre, être, entre, scientifique, pas, p...",[Pandémie de plagiat dans l'académie],"[de, et, des, la, un, en, le, vous, les, que]","[vous, nous, ou, faire, lettre, ceux, tous, in...",[un jeu qui vous permet de vous mettre dans la...
1,0,185,0_plagiat_de_le_du,Lutte contre le plagiat dans le monde académique,"[plagiat, de, le, du, la, les, des, il, et, une]","[plagieur, plagieurs, plagié, plagiats, plagia...",[Lutte contre le plagiat dans le monde académi...,"[plagiat, de, le, du, la, les, des, il, et, une]","[de, et, nous, étudiants, vous, académique, le...","[• enfin, pour tous ceux qui ne savent comment..."
2,1,64,1_fraude_chers_plagiat_action,Lutte contre la fraude académique,"[fraude, chers, plagiat, action, et, académiqu...","[fraude, fraudes, fraudeurs, plagiat, scientif...",[Lutte contre la fraude académique],"[fraude, chers, plagiat, action, et, académiqu...","[fraude, chers, plagiat, action, et, académiqu...",[• l' institut international de recherche et d...
3,2,53,2_université_doctorales_universitaires_écoles,Dispositifs d'intégrité dans les écoles doctor...,"[université, doctorales, universitaires, école...","[écoles, ecoles, universités, doctorales, univ...",[Dispositifs d'intégrité dans les écoles docto...,"[université, doctorales, universitaires, école...","[universitaires, écoles, et, nous, les, présid...","[face à cette demande, l’irafpa organise son a..."
4,3,47,3_journal_dans_publication_article,Problèmes de plagiat dans la publication scien...,"[journal, dans, publication, article, of, publ...","[publications, éditoriale, publication, une, a...",[Problèmes de plagiat dans la publication scie...,"[journal, dans, publication, article, of, publ...","[journal, article, deux, articles, auteurs, pu...",[le chercheur dont nous racontions la mésavent...
5,4,35,4_responsable_site_collaboratif_régulières,Reconnaissance pour contributions régulières v...,"[responsable, site, collaboratif, régulières, ...","[responsable, responsabilité, responsabilités,...",[Reconnaissance pour contributions régulières ...,"[responsable, site, collaboratif, régulières, ...","[responsable, site, contributions, vos, votre,...",[un tout grand merci pour vos contributions ré...
6,5,29,5_étudiants_déontologie_professeurs_et,Pratiques déontologiques en milieu étudiant,"[étudiants, déontologie, professeurs, et, leur...","[professeurs, étudiants, étudiant, une, lettre...",[Pratiques déontologiques en milieu étudiant],"[étudiants, déontologie, professeurs, et, leur...","[étudiants, déontologie, professeurs, et, leur...","[- enfin, pour tous ceux qui m’écrivent leur d..."
7,6,28,6_intégrité_sa_de_et,Enjeux de l'intégrité dans institutions et org...,"[intégrité, sa, de, et, institutions, organisa...","[organisationnelle, organisationnelles, organi...",[Enjeux de l'intégrité dans institutions et or...,"[intégrité, sa, de, et, institutions, organisa...","[et, institutions, organisations, les, nous, f...","[• certes, c’est au niveau global, structurel ..."
8,7,22,7_danger_que_vite_en,Préoccupation du danger et de la réponse,"[danger, que, vite, en, plusieurs, très, nombr...","[une, être, plaisir, un, de, plusieurs, en, vo...",[Préoccupation du danger et de la réponse],"[danger, que, vite, en, plusieurs, très, nombr...","[danger, que, plusieurs, très, outils, ont, ac...","[aussi, vos besoins étant souvent similaires, ..."
9,8,22,8_irafpa_luxembourg_institut_scientifique,Intégrité scientifique au Luxembourg,"[irafpa, luxembourg, institut, scientifique, m...","[institut, une, scientifique, scientifiques, n...",[Intégrité scientifique au Luxembourg],"[irafpa, luxembourg, institut, scientifique, m...","[irafpa, scientifique, missions, conseillers, ...",[au nom de tous les membres actifs de l’instit...


# Sélection des topics pertinents

In [52]:
# Visualize the documents in 2-dimensional space and show the titles on hover instead of the abstracts
fig = topic_model_2.visualize_documents(sentences_2,
                                reduced_embeddings=reduced_embeddings_2,
                                custom_labels=True,
                                # Tous les topics sauf 4, 7 et 17
                                topics=[topic if topic not in [-1, 11, 16] else None for topic in topics_2])
fig

In [53]:
fig.write_html("output/visualize_documents_all.html")

DTM

In [54]:
# La ligne suivante permet de lancer le DTM (Dynamic Topic Modeling) sur les données
topics_over_time = topic_model_2.topics_over_time(sentences_2, dates_2, evolution_tuning=True)

16it [00:00, 43.61it/s]


In [56]:
# Comme il s'agit d'un plotly, nous pouvons également tracer les étiquettes des sujets.
fig_dtm = topic_model_2.visualize_topics_over_time(topics_over_time, custom_labels=True, normalize_frequency=False, topics=[topic if topic not in [-1, 11, 16] else None for topic in topics_2])
fig_dtm.write_html("output/visualize_topics_over_time.html")
fig_dtm